In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.optimizers import Adam

In [2]:
# Set hyperparameters
max_features = 20000  # number of words to consider as features
max_len = 200  # maximum length of input sequences
batch_size = 32
epochs = 5

In [5]:
import numpy as np

In [6]:
data = np.load('imdb.npz')

In [7]:
# Load the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb.npz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)

In [8]:
import ssl
import os
import urllib.request
import numpy as np
from keras.datasets import imdb

# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context
os.environ['SSL_CERTIFICATE_VERIFY'] = '0'

# Load the IMDB dataset
urllib.request.urlretrieve("https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb.npz", "imdb.npz")
data = np.load("imdb.npz", allow_pickle=True)

# Print the shape of the training and test data
print("Training data shape:", data['x_train'].shape)
print("Training labels shape:", data['y_train'].shape)
print("Test data shape:", data['x_test'].shape)
print("Test labels shape:", data['y_test'].shape)

Training data shape: (25000,)
Training labels shape: (25000,)
Test data shape: (25000,)
Test labels shape: (25000,)


In [11]:
# Load the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 5s 0us/step


In [19]:
from keras.utils import pad_sequences
# Pad the sequences to the same length
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [20]:
# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          2560000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,609,473
Trainable params: 2,609,473
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [22]:
# Train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/5
782/782 [==============================] - 192s 239ms/step - loss: 0.4129 - accuracy: 0.8098 - val_loss: 0.3255 - val_accuracy: 0.8654
Epoch 2/5
782/782 [==============================] - 205s 262ms/step - loss: 0.2278 - accuracy: 0.9125 - val_loss: 0.3528 - val_accuracy: 0.8530
Epoch 3/5
782/782 [==============================] - 223s 285ms/step - loss: 0.1594 - accuracy: 0.9413 - val_loss: 0.3685 - val_accuracy: 0.8624
Epoch 4/5
782/782 [==============================] - 219s 280ms/step - loss: 0.1007 - accuracy: 0.9646 - val_loss: 0.4571 - val_accuracy: 0.8600
Epoch 5/5
782/782 [==============================] - 224s 286ms/step - loss: 0.0792 - accuracy: 0.9738 - val_loss: 0.5707 - val_accuracy: 0.8433


In [23]:
# Evaluate the model on test data
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 27s 34ms/step - loss: 0.5707 - accuracy: 0.8433
Test score: 0.5707300305366516
Test accuracy: 0.843280017375946
